In [1]:
import nltk
import unidecode
import pandas as pd
import os, re, string, gzip, itertools
import numpy as np
import tensorflow as tf
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from collections import defaultdict
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [2]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [3]:
my_dict = dict(zip(twenty_train.data , twenty_train.target))
train= pd.DataFrame.from_dict(my_dict, orient= 'index')
train.reset_index(inplace=True)
train.columns = ['Features', 'Target']
train.head()

,Features,Target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [4]:
my_dict = dict(zip(twenty_test.data , twenty_test.target))
test= pd.DataFrame.from_dict(my_dict, orient= 'index')
test.reset_index(inplace=True)
test.columns = ['Features', 'Target']
test.head()

,Features,Target
0,From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. ...,7
1,From: Rick Miller <rick@ee.uwm.edu>\nSubject: ...,5
2,From: mathew <mathew@mantis.co.uk>\nSubject: R...,0
3,From: bakken@cs.arizona.edu (Dave Bakken)\nSub...,17
4,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,19


In [5]:
df = train.append(test)
X = df.Features
y = df.Target
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [6]:
vectorizer = TfidfVectorizer()
training_features = vectorizer.fit_transform(X_train)    
test_features = vectorizer.transform(X_test)

In [7]:
D_train = xgb.DMatrix(training_features.toarray(), label=Y_train)
D_test = xgb.DMatrix(test_features.toarray(), label=Y_test)


KeyboardInterrupt: 

In [8]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 20} 

steps = 20  # The number of training iterations

In [9]:
model = xgb.train(param, D_train, steps)


NameError: name 'D_train' is not defined

In [11]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

NameError: name 'model' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)

In [ ]:
model.dump_model('dump.raw.txt')


In [ ]:
clf = xgb.XGBClassifier(max_depth = 4,
                       n_estimators = 50,
                       nthread = 4,
                       random_state = 314)



In [12]:
clf = xgb.XGBClassifier(max_depth = 3,
                       n_estimators = 25,
                       nthread = 4,
                       random_state = 314)




In [ ]:
clf = clf.fit(training_features.toarray(), Y_train)

/Users/satyammishra/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [ ]:
clf = LogisticRegression(C=1e2, solver = 'lbfgs', multi_class= 'multinomial')

In [ ]:
clf = GradientBoostingClassifier(max_depth = 3,
                                n_estimators = 50,
                                nthread = 4,
                                random_state = 314)